# Docker + Hadoop集群

为了使用Anaconda的Jupyter Lab和Jupyter Notebook进行基于Web接口的远程大数据及其分析实验，本教程介绍如何在Docker Hadoop集群的master节点上部署Anaconda数据科学实验环境。

## 1. Docker的安装

### Docker版本概览

Docker有三个版本，分别为：

1. **Docker Engine - Community（引擎社区版）** ：对于希望开始使用Docker并尝试使用基于容器的应用程序的个人开发人员和小团队来说，这是理想的选择。

2. **Docker Engine - Enterprise（引擎企业版）**：旨在用于企业开发具有安全性和企业级SLA的容器运行时。

3. **Docker Enterprise**：专为企业开发和IT团队而设计，他们可以在生产中大规模构建，发布和运行业务关键型应用程序。

### 获取Docker引擎社区版

1. 使用**共有云（Cloud）**：可以从亚马逊AWS和微软Azure注册（学生免费一年）账号，无需安装和配置，开箱即用。适合快速开展业务。
2. 安装Docker: 各平台下的安装请参考[官方文档](https://docs.docker.com/install/)
 

## 2. Docker用法

关于Docker的用法，请参考[官方入门教程](https://docs.docker.com/get-started/)或[菜鸟教程](https://www.runoob.com/docker/docker-tutorial.html)

## 3. 使用Docker部署Hadoop集群

这里以3台Docker主机为例，介绍如何使用Docker部署Hadoop集群

### 主机规划

3台主机：1个master、2个slaver/worker
ip地址使用docker默认的分配地址：

master：
主机名： master, ip地址： 172.17.0.2

slave：
主机名： slave1, ip地址： 172.17.0.3
主机名： slave2, ip地址： 172.17.0.4

### 创建Hadoop的Docker镜像

#### 获取centos镜像

In [ ]:
!docker pull centos

查看已下载的镜像:

In [2]:
!docker images

REPOSITORY                       TAG                       IMAGE ID            CREATED             SIZE
continuumio/anaconda3            latest                    cd83b04b8169        2 weeks ago         2.6GB
python                           latest                    34a518642c76        2 months ago        929MB
tomcat                           latest                    1c721f25f939        2 months ago        522MB
nginx                            latest                    62c261073ecf        2 months ago        109MB
redis                            latest                    d3e3588af517        2 months ago        95MB
ubuntu                           latest                    7698f282e524        2 months ago        69.9MB
alpine                           latest                    055936d39205        3 months ago        5.53MB
busybox                          latest                    64f5d945efcc        3 months ago        1.2MB
centos                           latest                

#### 在centos中安装SSH

1. 以centos7镜像为基础，使用**Dockerfile**构建一个带有SSH功能的centos

In [3]:
!cat Dockerfile

FROM centos
MAINTAINER baiwen1979@qq.com

RUN yum install -y openssh-server sudo
RUN sed -i 's/UsePAM yes/UsePAM no/g' /etc/ssh/sshd_config
RUN yum  install -y openssh-clients

RUN echo "root:abc123" | chpasswd
RUN echo "root   ALL=(ALL)       ALL" >> /etc/sudoers
RUN ssh-keygen -t dsa -f /etc/ssh/ssh_host_dsa_key
RUN ssh-keygen -t rsa -f /etc/ssh/ssh_host_rsa_key

RUN mkdir /var/run/sshd
EXPOSE 22
CMD ["/usr/sbin/sshd", "-D"]

这段代码的大意是：以 centos 镜像为基础，安装SSH的相关包，设置root用户的密码为abc123，并启动SSH服务

2. 执行构建镜像的命令，新镜像命名为 centos7-ssh

In [5]:
!docker build -t="centos7-ssh" .

Sending build context to Docker daemon  109.1kB
Step 1/12 : FROM centos
 ---> 9f38484d220f
Step 2/12 : MAINTAINER baiwen1979@qq.com
 ---> Running in bf91d86e1b9e
Removing intermediate container bf91d86e1b9e
 ---> 51be5aaf8b28
Step 3/12 : RUN yum install -y openssh-server sudo
 ---> Running in 066db83648dd
Loaded plugins: fastestmirror, ovl
Determining fastest mirrors
 * base: repos.cianhost.net
 * extras: repos.cianhost.net
 * updates: repos.cianhost.net
Resolving Dependencies
--> Running transaction check
---> Package openssh-server.x86_64 0:7.4p1-16.el7 will be installed
--> Processing Dependency: openssh = 7.4p1-16.el7 for package: openssh-server-7.4p1-16.el7.x86_64
--> Processing Dependency: fipscheck-lib(x86-64) >= 1.3.0 for package: openssh-server-7.4p1-16.el7.x86_64
--> Processing Dependency: libwrap.so.0()(64bit) for package: openssh-server-7.4p1-16.el7.x86_64
--> Processing Dependency: libfipscheck.so.1()(64bit) for package: openssh-server-7.4p1-16.el7.x86_64
---> Package sudo

Removing intermediate container b8bb2c66e37d
 ---> 4e01d375286c
Step 9/12 : RUN ssh-keygen -t rsa -f /etc/ssh/ssh_host_rsa_key
 ---> Running in 42be7549ed1e
Enter passphrase (empty for no passphrase): Enter same passphrase again: Generating public/private rsa key pair.
Your identification has been saved in /etc/ssh/ssh_host_rsa_key.
Your public key has been saved in /etc/ssh/ssh_host_rsa_key.pub.
The key fingerprint is:
SHA256:98etWxq+FCQou70BUE4y0wrhy8rUutaevGL5XjFhFsk root@42be7549ed1e
The key's randomart image is:
+---[RSA 2048]----+
|    .o=oo        |
|   .. EO.  .     |
|    ..=.o . . .  |
|   o +.o o   o   |
|  . + o S .   .  |
| o o   o = . . o |
|  +o  . . o . = o|
|  =o.o     o + = |
| o.=B.    .   *o |
+----[SHA256]-----+
Removing intermediate container 42be7549ed1e
 ---> ee9fa7e0b745
Step 10/12 : RUN mkdir /var/run/sshd
 ---> Running in 66d2530c279c
Removing intermediate container 66d2530c279c
 ---> 808737b271ff
Step 11/12 : EXPOSE 22
 ---> Running in 7d34521f99d6
Removing 

执行完成后，可以在镜像列表中看到:

In [7]:
!docker images

REPOSITORY                       TAG                       IMAGE ID            CREATED             SIZE
centos7-ssh                      latest                    76fb10914eb9        18 seconds ago      431MB
continuumio/anaconda3            latest                    cd83b04b8169        2 weeks ago         2.6GB
python                           latest                    34a518642c76        2 months ago        929MB
tomcat                           latest                    1c721f25f939        2 months ago        522MB
nginx                            latest                    62c261073ecf        2 months ago        109MB
redis                            latest                    d3e3588af517        2 months ago        95MB
ubuntu                           latest                    7698f282e524        2 months ago        69.9MB
alpine                           latest                    055936d39205        3 months ago        5.53MB
busybox                          latest                

#### 构建Hadoop镜像

我们需要运行3个centos容器模拟三台主机，为了避免在每个容器中重复安装和配置Hadoop,可以像构建SSH镜像一样，构建一个Hadoop镜像，然后运行3个Hadoop容器，这样非常方便。

In [8]:
!cat hadoop/Dockerfile

FROM centos7-ssh
ADD jdk-8u151-linux-x64.tar.gz /usr/local/
RUN mv /usr/local/jdk1.8.0_151 /usr/local/jdk1.8
ENV JAVA_HOME /usr/local/jdk1.8
ENV PATH $JAVA_HOME/bin:$PATH

ADD hadoop-3.1.0.tar.gz /usr/local
RUN mv /usr/local/hadoop-3.1.0 /usr/local/hadoop
ENV HADOOP_HOME /usr/local/hadoop
ENV PATH $HADOOP_HOME/bin:$PATH

RUN yum install -y which sudo

这段代码的含义是：基于centos7-ssh这个镜像，安装和配置JAVA和Hadoop的运行环境
【前提】：在Dockerfile所在目录下事先准备好jdk-8u151-linux-x64.tar.gz与hadoop-3.1.0.tar.gz

在hadoop目录下执行镜像构建命令，新镜像命名为centos-hadoop

In [ ]:
!docker build -t="centos-hadoop" .

需要依次在3台Docker容器中的/etc/hosts文件中添加3台主机的主机名和ip地址对应信息，而这些在重启容器后会被重置、覆盖。因此更好的办法是通过容器启动脚本docker run的–add-host参数将主机和ip地址的对应关系传入，容器在启动后会写入hosts文件中。如：

In [ ]:
!docker run --name master --add-host master:172.17.0.2 --add-host slave1:172.17.0.3 --add-host slave2:172.17.0.4 centos-hadoop

进入Docker容器，配置hadoop，在shell命令行中执行：
```
docker exec -it master bash
```
执行以下命令，配置hadoop免密登录：
```
ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod 0600 ~/.ssh/authorized_keys
```

#### hadoop部署

1. 在workers文件中定义工作节点
   在hadoop根目录下的etc/hadoop目录下新建workers文件，并添加工作节点主机信息。
   按照步骤一中的主机规划，工作节点主机为slave1和slave2两台主机。如：
   ```
   cat /usr/local/hadoop/etc/hadoop/workers
   slave1
   slave2
   ```
2. 修改配置文件信息
   * 在hadoop-env.sh中，添加JAVA_HOME信息
   ```
   cat /usr/local/hadoop/etc/hadoop/hadoop-env.sh |grep JAVA_HOME
   #JAVA_HOME=/usr/java/testing hdfs dfs -ls  
   export JAVA_HOME=/usr/local/jdk1.8
   ```
   * core-site.xml
   ```
   <configuration>
     <property>
       <name>fs.default.name</name>
       <value>hdfs://hadoop2:9000</value>
     </property>
     <property>
       <name>io.file.buffer.size</name>
       <value>131072</value>
     </property>
     <property>
       <name>hadoop.tmp.dir</name>
       <value>/home/hadoop/tmp</value>
       <description>Abase for other temporary directories.</description>
     </property>
   </configuration>
   ```
   * hdfs-site.xml
   ```
   <configuration>
     <property>
       <name>dfs.namenode.secondary.http-address</name>
       <value>master:9001</value>
       <description># 通过web界面来查看HDFS状态 </description>
     </property>
     <property>
       <name>dfs.namenode.name.dir</name>
       <value>/home/hadoop/dfs/name</value>
     </property>
     <property>
       <name>dfs.datanode.data.dir</name>
       <value>/home/hadoop/dfs/data</value>
     </property>
     <property>
       <name>dfs.replication</name>
       <value>2</value>
       <description># 每个Block有2个备份</description>
     </property>
     <property>
       <name>dfs.webhdfs.enabled</name>
       <value>true</value>
     </property>
   </configuration>
   ```
   * yarn-site.xml
   ```
   <configuration>
    <!-- Site specific YARN configuration properties -->
    <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
    </property>
    <property>
    <name>yarn.nodemanager.aux-services.mapreduce.shuffle.class</name>
    <value>org.apache.hadoop.mapred.ShuffleHandler</value>
    </property>
    <property>
    <name>yarn.resourcemanager.address</name>
    <value>master:8032</value>
    </property>
    <property>
    <name>yarn.resourcemanager.scheduler.address</name>
    <value>master:8030</value>
    </property>
    <property>
    <name>yarn.resourcemanager.resource-tracker.address</name>
    <value>master:8031</value>
    </property>
    <property>
    <name>yarn.resourcemanager.admin.address</name>
    <value>master:8033</value>
    </property>
    <property>
    <name>yarn.resourcemanager.webapp.address</name>
    <value>master:8088</value>
    </property>
    <property>
    <name>yarn.nodemanager.resource.memory-mb</name>
    <value>1024</value>
    </property>
    <property>
    <name>yarn.nodemanager.resource.cpu-vcores</name>
    <value>1</value>
    </property>
   </configuration>
   ```
   * mapred-site.xml
   ```
   <configuration>
    <property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
    </property>
    <property>
    <name>mapreduce.jobhistory.address</name>
    <value>master:10020</value>
    </property>
    <property>
    <name>mapreduce.jobhistory.webapp.address</name>
    <value>master:19888</value>
    </property>
   </configuration>
   ```
3. 编写启动/停止脚本
   * start-dfs.sh
   ```
   HDFS_DATANODE_USER=root
   #HADOOP_SECURE_DN_USER=hdfs
   HDFS_NAMENODE_USER=root
   HDFS_SECONDARYNAMENODE_USER=root
   HDFS_DATANODE_SECURE_USER=hdfs
   ```
   * start-yarn.sh
   ```
   YARN_RESOURCEMANAGER_USER=root
   HADOOP_SECURE_DN_USER=yarn
   YARN_NODEMANAGER_USER=root
   ```
注意：
以上步骤完成以后停止当前容器，并使用docker命令提交到一个新的镜像。使用新的镜像重新启动集群，这样集群每台机器都有相同的账户、配置和软件，无需再重新配置。即：

1. 停止容器
   ```
   docker stop master
   ```
2. 保存（提交）容器为新镜像
   ```
   docker commit master centos-hadoop:v1.0
   ```

#### 测试

1. 端口映射
   集群启动后，需要通过web界面观察集群的运行情况，因此需要将容器的端口映射到宿主主机的端口上，可以通过docker run命令的-p选项完成。比如：
   将yarn任务调度端口映射到宿主主机8088端口上：

   ```
   docker run -it -p 8088:8088 centos-hadoop:v1.0  
   ```
2. 从新镜像启动3个容器
   ```
   docker run --name master --add-host master:172.17.0.2 --add-host slave1:172.17.0.3 --add-host 
   salve2:172.17.0.4 -d -p 5002:22 -p 9870:9870 -p 8088:8088 -p 19888:19888 centos-hadoop:v1.0
    
   docker run --name slave1 --add-host master:172.17.0.2 --add-host slave1:172.17.0.3 --add-host 
   slave2:172.17.0.4 -d -p 5003:22 centos-hadoop:v1.0
    
   docker run --name slave2 --add-host master:172.17.0.2 --add-host slave1:172.17.0.3 --add-host 
   slave2:172.17.0.4 -d -p 5004:22 centos-hadoop:v1.0
   ```
3. 格式化HDFS
   进入到/usr/local/hadoop目录下,执行格式化命令:
   ```
   bin/hdfs namenode -format
   ```
   修改master中hadoop的一个配置文件etc/hadoop/slaves,删除原来的所有内容，修改为如下:
   ```
   slave1
   slave2
   ```
   在master中执行命令:
   ```
   scp  -rq /usr/local/hadoop   hadoop3:/usr/local
   scp  -rq /usr/local/hadoop   hadoop4:/usr/local
   ```
4. 在master主机上执行start-all.sh脚本启动集群
5. 通过web页面访问